In [1]:
import pandas as pd
import re
import string

In [2]:
df=pd.read_csv("train.csv")

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df.describe()

,id,label
count,20800.000000,20800.000000
mean,10399.500000,0.500625
std,6004.587135,0.500012
min,0.000000,0.000000
25%,5199.750000,0.000000
50%,10399.500000,1.000000
75%,15599.250000,1.000000
max,20799.000000,1.000000


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [6]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [7]:
df=df.fillna('')

In [8]:
df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [9]:
df.columns

Index(['id', 'title', 'author', 'text', 'label'], dtype='object')

In [10]:
df=df.drop(['id', 'title', 'author'], axis=1)

In [11]:
df.head()

,text,label
0,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,Ever get the feeling your life circles the rou...,0
2,"Why the Truth Might Get You Fired October 29, ...",1
3,Videos 15 Civilians Killed In Single US Airstr...,1
4,Print \nAn Iranian woman has been sentenced to...,1


In [12]:
def wordopt(text):
    text=text.lower()
    text=re.sub('\[.*?\]', '', text)
    text=re.sub("\\W"," ",text)
    text=re.sub('https?://\S+|www\.\S+','',text)
    text=re.sub('<.*?>+','',text)
    text=re.sub('[%s]' %re.escape(string.punctuation),'',text)
    text=re.sub('\n','',text)
    text=re.sub('\w*\d\w*','',text)
    return text

In [13]:
wordopt('Hi this is vashu')

'hi this is chando'

In [14]:
df['text']= df['text'].apply(wordopt)

In [15]:
x=df['text']

In [16]:
y=df['label']

In [17]:
y.shape

(20800,)

In [18]:
from sklearn.model_selection import train_test_split

In [19]:
x_train , x_test , y_train, y_test = train_test_split(x, y, test_size=0.20)

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [21]:
vect=TfidfVectorizer()

In [22]:
x_train=vect.fit_transform(x_train)
x_test=vect.transform(x_test)

In [23]:
x_test.shape

(4160, 155456)

In [24]:
from sklearn.linear_model import LogisticRegression

In [25]:
LR=LogisticRegression()

In [26]:
LR.fit(x_train,y_train)

LogisticRegression()

In [27]:
pred_lr=LR.predict(x_test)

In [28]:
pred_lr

array([0, 1, 0, ..., 0, 1, 1], dtype=int64)

In [29]:
LR.score(x_test, y_test)

0.9382211538461539

In [30]:
#gradient boosting classifier
from sklearn.ensemble import GradientBoostingClassifier

GB= GradientBoostingClassifier(random_state=0)
GB.fit(x_train,y_train)

GradientBoostingClassifier(random_state=0)

In [31]:
pred_gb = GB.predict(x_test)

In [32]:
pred_gb

array([0, 1, 0, ..., 0, 1, 1], dtype=int64)

In [33]:
GB.score(x_test,y_test)

0.9252403846153846

In [34]:
from sklearn.ensemble import RandomForestClassifier

RF=RandomForestClassifier(random_state=0)
RF.fit(x_train,y_train)

RandomForestClassifier(random_state=0)

In [35]:
pred_rf=RF.predict(x_test)

In [36]:
pred_rf

array([0, 1, 0, ..., 0, 1, 0], dtype=int64)

In [37]:
RF.score(x_test,y_test)

0.9074519230769231

In [38]:
from sklearn.tree import DecisionTreeClassifier

In [39]:
DT=DecisionTreeClassifier()

In [40]:
DT.fit(x_train, y_train)

DecisionTreeClassifier()

In [41]:
pred_dt=DT.predict(x_test)

In [42]:
pred_dt

array([0, 1, 0, ..., 0, 0, 0], dtype=int64)

In [43]:
DT.score(x_test, y_test)

0.8805288461538462

In [44]:
import pickle

In [45]:
pickle.dump(vect, open('vector.pkl', 'wb'))

In [46]:
pickle.dump(DT, open('DT.pkl', 'wb'))

In [47]:
pickle.dump(LR,open('LR.pkl','wb'))

In [48]:
pickle.dump(GB,open('GB.pkl','wb'))

In [49]:
pickle.dump(RF,open('RF.pkl','wb'))

In [50]:
vector_form=pickle.load(open('vector.pkl', 'rb'))

In [51]:
DT_model=pickle.load(open('DT.pkl', 'rb'))

In [52]:
LR_model=pickle.load(open('LR.pkl', 'rb'))

In [53]:
GB_model=pickle.load(open('GB.pkl', 'rb'))

In [54]:
RF_model=pickle.load(open('RF.pkl', 'rb'))

In [55]:
def output_label(n):
    if n==0:
        return "Fake News"
    elif n==1:
        return "Not a Fake News"

def fake_news(news):
    testing_news={"text":[news]}
    new_def_test=pd.DataFrame(testing_news)
    new_def_test["text"]=new_def_test["text"].apply(wordopt)
    new_x_test=new_def_test["text"]
    #new_xv_test=vectorization.transform(new_x_test)
    vector_form1=vector_form.transform(new_x_test)
    pred_LR=LR.predict(vector_form1)
    pred_DT=DT.predict(vector_form1)
    pred_GB=GB.predict(vector_form1)
    pred_RF=RF.predict(vector_form1)

    return print("\n\nLR Prediction: {} \nDT Prediction: {} \nGB Prediction: {} \nRF Prediction: {}".format(output_label(pred_LR[0]),
                                                                                                        output_label(pred_DT[0]),
                                                                                                        output_label(pred_GB[0]),
                                                                                                        output_label(pred_RF[0])))

In [56]:
news = str(input())
fake_news(news)

reliable
